In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

2.3.1
2.3.1
cuda


In [4]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

print("Sample Rate:", bundle.sample_rate)

print("Labels:", bundle.get_labels())

Sample Rate: 16000
Labels: ('-', '|', 'E', 'T', 'A', 'O', 'N', 'I', 'H', 'S', 'R', 'D', 'L', 'U', 'M', 'W', 'C', 'F', 'G', 'Y', 'P', 'B', 'V', 'K', "'", 'X', 'J', 'Q', 'Z')


In [7]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [5]:
model = bundle.get_model().to(device)

print(model.__class__)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [6]:
waveform, sample_rate = torchaudio.load("E:/project_summer/welcome_modified.wav")
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

In [8]:
with torch.inference_mode():
    emission, _ = model(waveform)

e:\anacon2\envs\mypyt\Lib\site-packages\torchaudio\models\wav2vec2\components.py:305: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [9]:
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
transcript = decoder(emission[0])

In [10]:
transcript

'THE|FOLLOWING|IS|AN|INTERVIEW|WITH|THE|PRES'

In [1]:
from pydub import AudioSegment
import os

# Load the audio file
audio = AudioSegment.from_file("E:\\chd\\putin_interview.wav")

# Define the number of chunks
num_chunks = 3024
chunk_length = len(audio) // num_chunks

# Create a directory to save chunks if it doesn't exist
output_dir = "E:\\chd\\audio_chunks"
os.makedirs(output_dir, exist_ok=True)

# Split the audio into chunks
audio_chunks = [audio[i*chunk_length:(i+1)*chunk_length] for i in range(num_chunks)]

# Save each chunk to the output directory with its index as the filename
for i, chunk in enumerate(audio_chunks):
    chunk.export(os.path.join(output_dir, f"chunk_{i}.wav"), format="wav")


e:\anacon2\envs\mypyt\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [12]:
import os

def list_files(directory):
    # Get the list of all files and directories in the specified directory
    files = os.listdir(directory)
    
    # Filter out only the files from the list
    files = [file for file in files if os.path.isfile(os.path.join(directory, file))]
    
    return files

# Example usage:
directory_path = 'E:/project_summer/data/audio_chunks'  # Replace with your directory path
files_list = list_files(directory_path)

print("Files in directory:")
for file_name in files_list:
    print(file_name)


Files in directory:
chunk_0.wav
chunk_1.wav
chunk_10.wav
chunk_100.wav
chunk_1000.wav
chunk_1001.wav
chunk_1002.wav
chunk_1003.wav
chunk_1004.wav
chunk_1005.wav
chunk_1006.wav
chunk_1007.wav
chunk_1008.wav
chunk_1009.wav
chunk_101.wav
chunk_1010.wav
chunk_1011.wav
chunk_1012.wav
chunk_1013.wav
chunk_1014.wav
chunk_1015.wav
chunk_1016.wav
chunk_1017.wav
chunk_1018.wav
chunk_1019.wav
chunk_102.wav
chunk_1020.wav
chunk_1021.wav
chunk_1022.wav
chunk_1023.wav
chunk_1024.wav
chunk_1025.wav
chunk_1026.wav
chunk_1027.wav
chunk_1028.wav
chunk_1029.wav
chunk_103.wav
chunk_1030.wav
chunk_1031.wav
chunk_1032.wav
chunk_1033.wav
chunk_1034.wav
chunk_1035.wav
chunk_1036.wav
chunk_1037.wav
chunk_1038.wav
chunk_1039.wav
chunk_104.wav
chunk_1040.wav
chunk_1041.wav
chunk_1042.wav
chunk_1043.wav
chunk_1044.wav
chunk_1045.wav
chunk_1046.wav
chunk_1047.wav
chunk_1048.wav
chunk_1049.wav
chunk_105.wav
chunk_1050.wav
chunk_1051.wav
chunk_1052.wav
chunk_1053.wav
chunk_1054.wav
chunk_1055.wav
chunk_1056.wav
chun

In [13]:
texts=[]
for file in files_list:
    waveform, sample_rate = torchaudio.load(f"E:/project_summer/data/audio_chunks/{file}")
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
    with torch.inference_mode():
        emission, _ = model(waveform)
    decoder = GreedyCTCDecoder(labels=bundle.get_labels())
    transcript = decoder(emission[0])
    texts.append(transcript)

In [16]:
texts

['THE|FOLLOWING|IS|AN|INTERVIEW|WITH|THE|PRES',
 'AN|OF|RUSSHOF|LADIMIR|PUTEN|',
 'OST|OBVIOUS|QUESTION|WHICH|IS|WHY|DID|YOU|DO|THIS|',
 'ADOPTED|TH|ORTHODOXY|A|E',
 'IS|IS|A|LARGE|NUMBER|OF|PEOPLE|YBEBI',
 'ITE|TE|LITAR|AND|IT|WAS|VERY|DIFFICULT|TO|SW',
 'AY|THIS|ELECTORATE|WHICH|HAD|THE|POSITIVE|',
 '|SHOW|TOWARDS|RUSSHASIT|QUITE|TIPISHOL|FIX|',
 'URIYANIKOVITCH|CAME|TO|POWER|AND|HA',
 'I|IN|RESTON||BEIN||FIRST|TIME|HE|WON|OFF|TE|PR',
 'AS|THEN|KUCHMA|THEY|ORGANIZED|THE|THIRD|RAU',
 'ONDE|WHICH|IS|NOT|PROVIDED|FOR|IN|THE|CONS',
 'TITUTION|OF|YOUR|CRAING|MUST|BE|ETTO|THIS|IS|',
 'COURDETAR|TE|JUST|IMAGINE|SOM',
 'STER|IN|CHRISTIANITY|CENTRLIZON|ERECIS|',
 "O|ONE|IN|THE|UNITED|STATES||WOULDN'T|LIKE|THE|OU",
 'COME|YES|IN|TWO|THOUSAND|FOURTEEN|',
 'AND|YET|BEFORE|THAT|',
 'NO|THIS|WAS|BEFORE|THAT|AFTER|',
 'EZDEN|KUCHMA|VICTORIAN|OVICWON|THE|L',
 'ECTIONS|HOWEVER|HIS|OPPONENTS|',
 'ID|NOT|RECOGNIZE|THAT|VICTORYOR|DE|U|S|',
 'UPPORTED|THE|OPPOSITION|AND|THE|THIRD|ROUND|',
 'SCATTU|RAP|AN

In [17]:
file_path = r"E:\project_summer\texts.txt"

# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    # Write each string from the list to the file, one per line
    for string in texts:
        file.write(string + "\n")
    
print(f"Strings have been saved to {file_path}")

Strings have been saved to E:\project_summer\texts.txt


In [18]:
def replace_pipes_with_spaces(text):
    return text.replace('|', ' ')

# Example usage:
text = "THE|FOLLOWING|IS|AN|INTERVIEW|WITH|THE|PRES"
result = replace_pipes_with_spaces(text)
print(result)


THE FOLLOWING IS AN INTERVIEW WITH THE PRES


In [20]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the dataset
# data = pd.read_csv('sentences.csv')
sentences = [replace_pipes_with_spaces(item) for item in texts]

# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(sentences)

# Save embeddings to a file for further use
np.save('embeddings.npy', embeddings)


e:\anacon2\envs\mypyt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
embeddings[0]

array([-7.42278472e-02,  1.38715133e-01,  4.33864035e-02,  9.53941327e-03,
        1.03020826e-02,  7.35115856e-02,  2.54281890e-02,  2.17058342e-02,
       -1.88467428e-02,  4.06900235e-02, -2.77303420e-02,  2.90727168e-02,
       -5.50810732e-02, -7.44152740e-02, -7.59306774e-02, -9.97447036e-03,
        9.77737978e-02, -9.48362723e-02, -1.54412659e-02,  2.59865467e-02,
       -4.32665199e-02, -2.42114887e-02,  2.19918098e-02,  4.87505384e-02,
        3.16648036e-02,  1.94319384e-03,  4.35861119e-04,  6.68247230e-03,
        6.22695945e-02,  4.58670743e-02,  3.43116955e-03, -3.42065170e-02,
        1.10349707e-01,  5.85297085e-02, -3.94177996e-02,  6.77843466e-02,
        9.52500924e-02, -1.19562466e-02,  6.31892532e-02,  1.84966642e-02,
        1.05395890e-03, -9.57159325e-02, -1.38647705e-02,  4.19445075e-02,
       -4.45200801e-02,  1.45932031e-03,  2.60545164e-02,  3.50528583e-02,
       -5.21483757e-02, -1.12036169e-02, -5.83827682e-02, -5.54148145e-02,
        3.33681516e-02, -

In [22]:
!pip install chromadb



^C


  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------------------------------------ --- 61.4/67.3 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 67.3/67.3 kB 919.7 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ---------------------------------------- 51.6/51.6 kB 2.6 MB/s eta 0:00:00
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached fastapi_cli-0.0.4-py3-none-any.whl.metadata (7.0 kB)
  Using cached email_va

In [23]:
search_query ="the presedent"

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
query_embedding = model.encode([search_query])

# Compute similarity for original embeddings
similarities_original = cosine_similarity(query_embedding, embeddings)
top_10_indices_original = similarities_original.argsort()[0][-10:][::-1]
top_10_sentences_original = [sentences[i] for i in top_10_indices_original]

In [25]:
top_10_sentences_original

['OR DOES IT NOT MATTER WHO THE PRESIDERT IS ',
 'THE FORMER PRESIDENT CONDELISE SE ',
 'ALSO THE PRESIDENT OF THE UNITED STATES ',
 'MOREOVER THE PRESIDENT YOU RE',
 'HIMSELF PRESIDENT THEN HE IS RECOGNIS',
 ' FOR THE FIRST TWENTY TWO YEARS AS PRESIDENT ',
 'LL OTHER OFFICIALS AND THEN PRESIDENT HIMS',
 'AGINED COCTHE PRESIDENT OF YOU CRAN HIM',
 'THE FOLLOWING IS AN INTERVIEW WITH THE PRES',
 'N PARTY RAGUONS THE PARTY OF THE PRESIDENT ']